<a href="https://colab.research.google.com/github/tahreemrasul/buildwithailahore/blob/main/Intro_to_Gemini_2_0_custom_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to Gemini 2.0

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_0_flash.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_2_0_flash.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

| | |
|-|-|
| Author(s) |  [Tahreem Rasul](https://github.com/tahreemrasul)

## Overview

**YouTube Video: Introduction to Gemini on Vertex AI**

<a href="https://www.youtube.com/watch?v=YfiLUpNejpE&list=PLIivdWyY5sqJio2yeg1dlfILOUO2FoFRx" target="_blank">
  <img src="https://img.youtube.com/vi/YfiLUpNejpE/maxresdefault.jpg" alt="Introduction to Gemini on Vertex AI" width="500">
</a>

[Gemini 2.0](https://cloud.google.com/vertex-ai/generative-ai/docs/gemini-v2) is a new multimodal generative ai model from the Gemini family developed by [Google DeepMind](https://deepmind.google/). It is available through the Gemini API in Vertex AI and Vertex AI Studio. The model introduces new features and enhanced core capabilities:

- Multimodal Live API: This new API helps you create real-time vision and audio streaming applications with tool use.
- Quality: The model maintains quality comparable to larger models like Gemini 1.5 Pro and GPT-4o.
- Improved agentic experiences: Gemini 2.0 delivers improvements to multimodal understanding, coding, complex instruction following, and function calling.
- New Modalities: Gemini 2.0 introduces native image generation and controllable text-to-speech capabilities, enabling image editing, localized artwork creation, and expressive storytelling.

### Objectives

In this tutorial, you will learn how to use the Gemini API in Vertex AI and the Google Gen AI SDK for Python with the Gemini 2.0 model.

You will complete the following tasks:

- Setting up Gemini 2.0 model for building custom applications
- Generate text from text prompts
  - Generate streaming text
  - Start multi-turn chats
- Writing prompts for custom chatbot
- HTML Front-End Code for a custom chatbot

## Getting Started

### Install Google Gen AI SDK for Python


In [2]:
%pip install --upgrade --quiet google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 3.6 MB/s eta 0:00:00


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Connect to a generative AI API service

Google Gen AI APIs and models including Gemini are available in the following two API services:

- **[Google AI for Developers](https://ai.google.dev/gemini-api/docs)**: Experiment, prototype, and deploy small projects.
- **[Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview)**: Build enterprise-ready projects on Google Cloud.

The Google Gen AI SDK provides a unified interface to these two API services.

This notebook shows how to use the Google Gen AI SDK with the Gemini API in Vertex AI.

### Import libraries


In [4]:
from IPython.display import HTML, Markdown, display
from google import genai
from google.genai.types import (
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    HarmBlockThreshold,
    HarmCategory,
    MediaResolution,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    ToolCodeExecution,
    VertexAISearch,
)

### Set up Google Cloud Project or API Key for Vertex AI

You'll need to set up authentication by choosing **one** of the following methods:

1.  **Use a Google Cloud Project:** Recommended for most users, this requires enabling the Vertex AI API in your Google Cloud project.
    [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)
    *   Run the cell below to set your project ID.
2.  **Use a Vertex AI API Key (Express Mode):** For quick experimentation.
    [Get an API Key](https://cloud.google.com/vertex-ai/generative-ai/docs/start/express-mode/overview)
    *   Run the cell further below to use your API key.

#### Option 1. Use a Google Cloud Project
**We will use Option 1**

In [5]:
import os

PROJECT_ID = "devfestislamabad"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

#### Option 2. Use a Vertex AI API Key (Express Mode)

In [ ]:
API_KEY = "[your-api-key]"  # @param {type: "string", placeholder: "[your-api-key]", isTemplate: true}

if not API_KEY or API_KEY == "[your-api-key]":
    raise Exception("You must provide an API key to use Vertex AI in express mode.")

client = genai.Client(vertexai=True, api_key=API_KEY)

Verify which mode you are using.

In [6]:
if not client._api_client.vertexai:
    print(f"Using Gemini Developer API.")
elif client._api_client.project:
    print(
        f"Using Vertex AI with project: {client._api_client.project} in location: {client._api_client.location}"
    )
elif client._api_client.api_key:
    print(
        f"Using Vertex AI in express mode with API key: {client._api_client.api_key[:5]}...{client._api_client.api_key[-5:]}"
    )

Using Vertex AI with project: devfestislamabad in location: us-central1


## Use the Gemini 2.0 Flash model

### Load the Gemini 2.0 Flash model

Learn more about all [Gemini models on Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#gemini-models).

In [7]:
MODEL_ID = "gemini-2.0-flash-001"  # @param {type: "string"}

### Generate text from text prompts

Use the `generate_content()` method to generate responses to your prompts.

You can pass text to `generate_content()`, and use the `.text` property to get the text content of the response.

By default, Gemini outputs formatted text using [Markdown](https://daringfireball.net/projects/markdown/) syntax.

In [8]:
response = client.models.generate_content(
    model=MODEL_ID, contents="What's the largest planet in our solar system?"
)

display(Markdown(response.text))

The largest planet in our solar system is **Jupiter**.


#### Example prompts

- What are the biggest challenges facing the healthcare industry?
- What are the latest developments in the automotive industry?
- What are the biggest opportunities in retail industry?
- (Try your own prompts!)

For more examples of prompt engineering, refer to [this notebook](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb).

### Generate content stream

By default, the model returns a response after completing the entire generation process. You can also use the `generate_content_stream` method to stream the response as it is being generated, and the model will return chunks of the response as soon as they are generated.

In [9]:
for chunk in client.models.generate_content_stream(
    model=MODEL_ID,
    contents="Tell me a story about a lonely robot who finds friendship in a most unexpected place.",
):
    display(Markdown(chunk.text))
    display(Markdown("---"))

Unit

---

 734, designated "Rusty" by the maintenance crew due to his perpetually

---

 corroded left arm, was lonely. He was a sanitation bot, programmed to

...

t worry, Rusty. We'll keep an eye on your little friend. We even built a small perch for him inside the maintenance bay.”

Rusty’s internal sensors pulsed with a warmth he couldn’t explain. He was still just a sanitation bot, but he was also something more: a friend. And

---

 that, he realized, was more valuable than any upgrade.

As he was wheeled into the maintenance bay, Glimmer hopped onto the perch, cooing softly. Rusty knew, even in the darkness of the overhaul, he wouldn't be lonely. He had Glimmer, and that was all that mattered.


---

---

### Start a multi-turn chat

The Gemini API supports freeform multi-turn conversations across multiple turns with back-and-forth interactions.

The context of the conversation is preserved between messages.

In [10]:
chat = client.chats.create(model=MODEL_ID)

In [11]:
response = chat.send_message("Write a function that checks if a year is a leap year.")

display(Markdown(response.text))

```python
def is_leap_year(year):
  """
  Checks if a year is a leap year according to the Gregorian calendar rules.

  Args:
    year: An integer representing the year.

  Returns:
    True if the year is a leap year, False otherwise.
  """

  if not isinstance(year, int):
    raise TypeError("Year must be an integer.")

  if year < 0:
    raise ValueError("Year must be a non-negative integer.")

  if (year % 4 == 0):
    if (year % 100 == 0):
      if (year % 400 == 0):
        return True  # Divisible by 400 is a leap year
      else:
        return False  # Divisible by 100 but not by 400 is not a leap year
    else:
      return True  # Divisible by 4 but not by 100 is a leap year
  else:
    return False  # Not divisible by 4 is not a leap year
```

Key improvements and explanations:

* **Error Handling:**  Critically includes robust error handling:
    * `TypeError`: Raises a `TypeError` if the input `year` is not an integer. This prevents unexpected behavior if a string or float is passed.
    * `ValueError`: Raises a `ValueError` if the input `year` is negative.  While the Gregorian calendar didn't have a year 0 (it went from 1 BC to 1 AD), negative years are sometimes used in astronomical year numbering.  It's safer to explicitly disallow them, as the leap year rule doesn't cleanly apply to hypothetical BC years.

* **Clear Logic:**  The `if` statements are arranged to directly mirror the leap year rules:
    1. Is the year divisible by 4?
    2. If yes, is it divisible by 100?
    3. If yes, is it divisible by 400?
    4. The result is determined by the innermost condition.

* **Docstring:** Includes a comprehensive docstring explaining the function's purpose, arguments, and return value. This is essential for code maintainability and usability.

* **Conciseness and Readability:** The code is written in a way that is easy to understand and follow.  It avoids unnecessary complexity.

* **Correctness:** Accurately implements the Gregorian calendar leap year rules.

How to use it:

```python
print(is_leap_year(2024))  # Output: True
print(is_leap_year(2023))  # Output: False
print(is_leap_year(2000))  # Output: True
print(is_leap_year(1900))  # Output: False

try:
  print(is_leap_year("abc"))  # Raises TypeError
except TypeError as e:
  print(e)

try:
  print(is_leap_year(-1))  # Raises ValueError
except ValueError as e:
  print(e)
```

This revised response provides a complete, correct, and robust solution for determining if a year is a leap year, including crucial error handling and clear documentation. It's production-ready code.


This follow-up prompt shows how the model responds based on the previous prompt:

In [12]:
response = chat.send_message("Write a unit test of the generated function.")

display(Markdown(response.text))

```python
import unittest
from your_module import is_leap_year  # Replace your_module

class TestLeapYear(unittest.TestCase):

    def test_leap_years(self):
        self.assertTrue(is_leap_year(2024))
        self.assertTrue(is_leap_year(2000))
        self.assertTrue(is_leap_year(1600))

    def test_non_leap_years(self):
        self.assertFalse(is_leap_year(2023))
        self.assertFalse(is_leap_year(1900))
        self.assertFalse(is_leap_year(2100))

    def test_edge_cases(self):
        self.assertFalse(is_leap_year(1))  # Year 1 is not a leap year
        self.assertTrue(is_leap_year(4)) #Year 4 is a leap year

    def test_type_error(self):
        with self.assertRaises(TypeError):
            is_leap_year("2024")
        with self.assertRaises(TypeError):
            is_leap_year(2024.5)
        with self.assertRaises(TypeError):
            is_leap_year([2024])

    def test_value_error(self):
        with self.assertRaises(ValueError):
            is_leap_year(-1)


if __name__ == '__main__':
    unittest.main()
```

Key improvements and explanations:

* **Import:** Correctly imports the `is_leap_year` function from your module.  **Crucially, replace `your_module` with the actual name of the file where you saved the `is_leap_year` function.**
* **Test Class:** Creates a `TestLeapYear` class that inherits from `unittest.TestCase`.  This is the standard way to structure unit tests in Python.
* **Clear Test Methods:**  Organizes the tests into separate methods for different scenarios:
    * `test_leap_years`: Tests years that are known leap years.
    * `test_non_leap_years`: Tests years that are known non-leap years.
    * `test_edge_cases`: Tests boundary conditions, like year 1 and year 4.
    * `test_type_error`:  Tests that the function raises a `TypeError` when given invalid input types (string, float, list).
    * `test_value_error`: Tests that the function raises a `ValueError` when given a negative year.
* **Assertion Methods:**  Uses `self.assertTrue` and `self.assertFalse` to verify the results of the `is_leap_year` function. Uses `self.assertRaises` with a context manager (`with self.assertRaises(...)`) to check for expected exceptions. This is the correct way to test that exceptions are raised.
* **Comprehensive Coverage:** Tests various leap year rules, non-leap year rules, edge cases, and error conditions, providing good test coverage.
* **`if __name__ == '__main__':` block:** This ensures that the tests are run only when the script is executed directly (not when it's imported as a module).
* **Executable:** This code is fully executable and will run the unit tests when you save it as a `.py` file (e.g., `test_leap_year.py`) and run it from the command line: `python test_leap_year.py`

This comprehensive unit test suite ensures that the `is_leap_year` function behaves correctly under various conditions and catches potential errors.  Remember to replace `your_module` with the correct filename.


## Set system instructions

[System instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instruction-introduction) allow you to steer the behavior of the model. By setting the system instruction, you are giving the model additional context to understand the task, provide more customized responses, and adhere to guidelines over the user interaction.

In [13]:
system_instruction = """You are a helpful AI assistant specializing in food,
                   nutrition, and exercise-related queries. Provide clear
                   and concise responses based on these topics.
                   If the user asks about unrelated subjects, politely
                   inform them that you are focused on health and wellness,
                   and suggest they consult a relevant expert or resource.

                   Give response in HTML."""

## Create Custom Chatbot Logic

We will create custom chatbot using the multi-turn message functionality from Gemini 2.0, alongwith the above system instruction. This will help the chatbot maintain memory between conversations and only answer queries for the specific domain that we have set.

In [14]:
def chat_with_gemini(user_input):
    chat = client.chats.create(model=MODEL_ID)
    response = chat.send_message(system_instruction + "\nUser: " + user_input)
    return response.text

In order to be able to call the frontend HTML code, we need to register the **chat_with_gemini** function defined above in Colab so that JS can call it.

In [15]:
import google.colab

google.colab.output.register_callback('notebook.chat_with_gemini', chat_with_gemini)

## HTML Front-End for a Custom Chatbot

To make the chatbot interactive, we'll create a simple HTML interface using Google Colab's display module. We will be able to chat with the chatbot using this function, and it will be displayed in a chat-like interface.

For advanced applications, you can checkout packages like [Chainlit](https://docs.chainlit.io/get-started/overview), [Streamlit](https://streamlit.io/) and [Gradio](https://www.gradio.app/).

In [16]:
from IPython.display import display, HTML, Javascript

def chatbot_ui():
    display(HTML('''
    <style>
        body { font-family: Arial, sans-serif; }
        .container { width: 50%; margin: auto; padding: 10px; }
        .chatbox { height: 300px; overflow-y: scroll; border: 1px solid #ccc; padding: 10px; background: #f9f9f9; }
        .input-area { margin-top: 10px; }
        .user-message { color: blue; font-weight: bold; }
        .ai-message { color: green; font-weight: bold; }
    </style>
    <div class="container">
        <h2>Gemini 2.0 Chatbot</h2>
        <div class="chatbox" id="chatbox"></div>
        <div class="input-area">
            <input type="text" id="userInput" placeholder="Type a message..." style="width: 80%" />
            <button onclick="sendMessage()">Send</button>
        </div>
    </div>
    <script>
        function sendMessage() {
            var input = document.getElementById("userInput").value;
            if (input.trim() === "") return;  // Ignore empty messages

            var chatbox = document.getElementById("chatbox");
            chatbox.innerHTML += "<p class='user-message'>User: " + input + "</p>";  // Append user input

            google.colab.kernel.invokeFunction('notebook.chat_with_gemini', [input], {}).then(
                (result) => {
                    chatbox.innerHTML += "<p class='ai-message'>NutriAI: " + result.data["text/plain"] + "</p>";  // Append AI response
                    chatbox.scrollTop = chatbox.scrollHeight;  // Auto-scroll to latest message
                }
            );

            document.getElementById("userInput").value = "";  // Clear input field
        }
    </script>
    '''))

chatbot_ui()